<a href="https://colab.research.google.com/github/colinmcnamara/austin_langchain/blob/main/labs/LangChain_103/rag_ollama_llava_drive/103-1-ollama-bakllava-gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Create requirements.txt file with project dependencies

In [8]:
%%writefile requirements.txt
langchain-core 
langchain-community 
streamlit 
google-api-python-client 
google-auth-httplib2 
google-auth-oauthlib

Overwriting requirements.txt


### Create **mygdrive** module with *MyGDrive* class to encapsulate our Google Drive API interactions

In [9]:
%%writefile mygdrive.py
import base64
import io
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account


class MyGDrive:
    def __init__(self, service_account_json_key):
        scope = ['https://www.googleapis.com/auth/drive']
        # service_account_json_key = 'service-account-key.json'
        credentials = service_account.Credentials.from_service_account_file(
                                      filename=service_account_json_key,
                                      scopes=scope)
        self.service = build('drive', 'v3', credentials=credentials)

    def get_files(self):
        results = (
                self.service.files()
                .list(pageSize=1000,
                      fields="nextPageToken, "
                      "files(id, name, mimeType, size, modifiedTime, parents)",
                      q='mimeType != "application/vnd.google-apps.folder"')
                .execute()
                )
        # get the results
        return results.get('files', [])

    def get_file_contents(self, fileId: str, encoded: bool = True):
        try:
            request_file = self.service.files().get_media(fileId=fileId)
            file = io.BytesIO()
            downloader = MediaIoBaseDownload(file, request_file)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
            file_retrieved = file.getvalue()
            if encoded:
                return base64.b64encode(file_retrieved).decode()
            else:
                return file_retrieved
        except HttpError as error:
            print(F'An error occurred: {error}')


Overwriting mygdrive.py


### Create our Streamlit App

In [ ]:
%%writefile streamlitapp.py
import base64
import streamlit as st
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import AIMessage, HumanMessage
from mygdrive import MyGDrive
from operator import itemgetter


def bind_and_run_llm(payload):
    image = payload["image"]
    prompt = payload["prompt"]
    bound = llm.bind(images=[image])
    return bound.invoke(prompt)


@st.cache_data
def get_service(service_account_key):
    return MyGDrive(service_account_key)


@st.cache_data
def get_files(_service: MyGDrive):
    return _service.get_files()


@st.cache_data
def get_file_contents(_service: MyGDrive, fileId: str, encoded: bool):
    return _service.get_file_contents(fileId, encoded)


st.title("Multimodal Chat")
llm = Ollama(model="bakllava")

image_prompt = PromptTemplate.from_template("{image}")
prompt = PromptTemplate.from_template("{question}")

chain = (
    {"image": itemgetter("image"), "prompt": prompt} |
    RunnableLambda(bind_and_run_llm)
)


if "messages" not in st.session_state:
    st.session_state["messages"] = [
            (AIMessage(content="How can I help you?"), False)
            ]

if "uploaded_file" not in st.session_state:
    st.session_state["uploaded_file"] = None

for msg in st.session_state.messages:
    if not msg[1]:
        st.chat_message(msg[0].type).write(msg[0].content)
    else:
        st.chat_message(msg[0].type).image(msg[0].data, width=200)


def on_upload_image(fileName, fileContents):
    st.session_state.uploaded_file = fileName
    st.session_state.uploaded_file_contents = fileContents
    st.session_state.messages.append(
            (HumanMessage(
                content=fileName,
                data=fileContents
                ), True))
    st.chat_message("user").image(fileContents, width=200)


with st.sidebar:
    if uploaded_file := st.file_uploader("Upload an image file",
                                         type=["jpg", "png"]):
        on_upload_image(uploaded_file.name, uploaded_file.getvalue())

    if gcloud_service_key_file := st.file_uploader(
            "Upload your Google Cloud Service Account Key file "
            "to access images from your Google Drive", type=["json"]
            ):
        with open(gcloud_service_key_file.name, "wb") as f:
            f.write(gcloud_service_key_file.getvalue())
        st.session_state["gcloud_service_account"] = gcloud_service_key_file.name

if "gcloud_service_account" in st.session_state.keys():
    mydrive = get_service(st.session_state["gcloud_service_account"])
    for item in get_files(mydrive):
        if item["mimeType"][0:5] == "image":
            fileId = item["id"]
            fileName = item["name"]
            contents = get_file_contents(mydrive, fileId, False)
            st.sidebar.image(contents, width=100)
            if st.sidebar.button("Use", key=fileId):
                on_upload_image(fileName, contents)

if prompt := st.chat_input():
    st.session_state.messages.append((HumanMessage(content=prompt), False))
    st.chat_message("human").write(prompt)

    response = ""
    if st.session_state.uploaded_file_contents is not None:
        data = st.session_state.uploaded_file_contents
        b64 = base64.b64encode(data).decode()

        response = chain.invoke({"question": prompt, "image": b64})
    else:
        response = "Please upload an image first"

    st.session_state.messages.append((AIMessage(content=response), False))
    st.chat_message("assistant").write(response)


### Prepare Google Drive

1. Go to [Google Cloud Console](https://console.cloud.google.com) and create a new Google Cloud Project
2. From search bar on the dashboard, search for `Google Drive API`
3. Click `Google Drive API` from the search results list (under Marketplace) and click `Enable` to add it to you project
4. From `IAM & Admin`, go to `Service Accounts` and create a new Service Account
    * Download the key in json format and save it in a secure place
    * Make a note of the service account name (in email address format)
5. On you Google Drive, create a new folder and upload a few image files in it
6. Share the newly created Google Drive folder with the email address of the service account you created in step 2 

### Download and run ollama

Below, we:
1. download the ollama binary
2. make it executable
3. start ollama in the background
4. download the hosted bakllava model

In [ ]:
%pip install -r requirements.txt

In [3]:
%%capture
!curl -L https://ollama.ai/download/ollama-linux-amd64 -o ollama
!chmod +x ollama
!./ollama serve &>/content/ollama_logs.txt &
!./ollama pull bakllava

### Start and background streamlit app

In [ ]:
!streamlit run streamingapp.py &>/content/logs.txt &

## Find the IP of your instance

In [5]:
!curl ipv4.icanhazip.com
!echo "Copy this IP into the webpage that opens below"

35.227.3.82
Copy this IP into the webpage that opens below


## Expose the Streamlit app on port 8501

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 3.73s
your url is: https://common-mails-doubt.loca.lt
